In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import tensorflow as tf 
from tqdm import tqdm

In [2]:
# Define the number of classes
num_classes = len(os.listdir('C:/Users/AlessandroLaVeglia/repo_git/Vision-Exam/immagini'))

# Load MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
# Add your own fully connected layers for classification
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
# Set up data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)


In [6]:
# Create a tf.data.Dataset for efficient data loading
train_dataset = tf.data.Dataset.from_generator(
    lambda: datagen.flow_from_directory(
        'C:/Users/AlessandroLaVeglia/repo_git/Vision-Exam/immagini',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    ),
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
)

# Train the model using tf.data.Dataset
model.fit(
    train_dataset,
    steps_per_epoch=len(data_generator),
    epochs=10
)

Found 8975 images belonging to 9 classes.
Epoch 1/10



103/281 [=========>....................] - ETA: 4:52 - loss: 0.6788 - accuracy: 0.7952

KeyboardInterrupt: 